### Using chromadb

In [18]:
from chromaClient import ChromaClient

In [19]:
chroma_client = ChromaClient()

In [20]:
chroma_client.collection_list

[Collection(name=wf_schema), Collection(name=langchain)]

In [21]:
from chromadb.utils import embedding_functions

In [22]:
base_model = "BM-K/KoSimCSE-roberta-multitask"
emb_func = embedding_functions.SentenceTransformerEmbeddingFunction(model_name=base_model, normalize_embeddings=True)

In [26]:
chroma_client.connect_collection(coll_name='langchain', emb_model=base_model)

In [27]:
chroma_client.collection.count()

0

In [8]:
chroma_client.semantic_search(['배고파ㅏㅏㅏㅏ'], 3)

{'ids': [['1d86f62f-8243-11ee-bb56-9822ef80fc28',
   'd2afb478-8242-11ee-a3e0-9822ef80fc28',
   '37adcb50-8247-11ee-91a7-9822ef80fc28']],
 'distances': [[0.6858786344528198, 0.6905706524848938, 0.6916413307189941]],
 'metadatas': [[None, None, None]],
 'embeddings': None,
 'documents': [['-',
   '위기 상황',
   '대상\n\n기준 중위소득 75% 이하의 북한이탈주민(세대)으로, 다음 사유로 인해 생계 안정이 필요한 경우:\n\n주소득자가 사망, 가출, 행방불명, 구금시설에 수용되는 등의 사유로 소득을 상실한 경우\n\n주소득자 또는 부소득자의 실직, 휴업, 폐업 또는 사업장의 화재, 중한 질병이나 부상 등으로 소득 상실 또는 소득활동이 중단된 경우']],
 'uris': None,
 'data': None}

### Using vectorstores

In [25]:
from langchain.vectorstores.chroma import Chroma
from langchain.embeddings import SentenceTransformerEmbeddings as STE

In [2]:
def _device_check():
    ''' for check cuda availability '''
    import torch
    device = "cuda" if torch.cuda.is_available() else "cpu"
    # torch.backends.mps.is_available()
    return device

In [3]:
embedding = STE(
    model_name="BM-K/KoSimCSE-roberta-multitask", 
    model_kwargs={'device': _device_check()},
    encode_kwargs={'normalize_embeddings': True},
    )

In [6]:
from langchain.chains import RetrievalQA

In [20]:
vector_store = Chroma(collection_name='langchain', persist_directory="workspace/chroma_storage", embedding_function=embedding)

In [22]:
rt = vector_store.as_retriever(search_type='mmr')

In [23]:
q = "졸라 배고파ㅏㅏㅏ"

In [24]:
res = rt.get_relevant_documents(q)

In [25]:
res

[Document(page_content='식비 \n  1일 5시간 이상(월평균 100시간 이상)인 과정 참여자 중 훈련기관에서 식사를 제공하지 않는 경우 월 최대 6만 6,000 원(3,300 원일) 지급  통학생 중 개인적 사정(식이조절 등으로 인한 개별 도시락 지참 등)으로 식사 제공 대신 식비 지급을 희망할 경우 월 최대 6만 6,000 원(3,300 원일) 지급', metadata={'source': 'markdowndb\\07_장애인_지원\\07_장애인_직업능력개발_운영(훈련수당).md'}),
 Document(page_content='위기 상황', metadata={'source': 'markdowndb\\01_생계_지원\\01_긴급복지_지원제도.md'}),
 Document(page_content='실업급여\n\n대상', metadata={'source': 'markdowndb\\02_취업_지원\\02_실업급여.md'}),
 Document(page_content='또는 건물에서 생활하기 곤란한 경우  주소득자 또는 부소득자의 휴업, 폐업 또는 사업장의 화재 등으로 영업이 곤란하게 된 경우  주소득자 또는 부소득자의 실직으로 소득을 상실한 경우  지자체 조례로 정한 사유가 발생한 경우  소득활동 미미(가구원 간호, 간병, 양육), 기초수급 중지, 미결정, 수도·가스 중단, 사회보험료·주택임차료 체납 등  관할', metadata={'source': 'markdowndb\\01_생계_지원\\01_긴급복지_지원제도.md'})]

In [26]:
vector_store.max_marginal_relevance_search(q)

[Document(page_content='식비 \n  1일 5시간 이상(월평균 100시간 이상)인 과정 참여자 중 훈련기관에서 식사를 제공하지 않는 경우 월 최대 6만 6,000 원(3,300 원일) 지급  통학생 중 개인적 사정(식이조절 등으로 인한 개별 도시락 지참 등)으로 식사 제공 대신 식비 지급을 희망할 경우 월 최대 6만 6,000 원(3,300 원일) 지급', metadata={'source': 'markdowndb\\07_장애인_지원\\07_장애인_직업능력개발_운영(훈련수당).md'}),
 Document(page_content='위기 상황', metadata={'source': 'markdowndb\\01_생계_지원\\01_긴급복지_지원제도.md'}),
 Document(page_content='실업급여\n\n대상', metadata={'source': 'markdowndb\\02_취업_지원\\02_실업급여.md'}),
 Document(page_content='또는 건물에서 생활하기 곤란한 경우  주소득자 또는 부소득자의 휴업, 폐업 또는 사업장의 화재 등으로 영업이 곤란하게 된 경우  주소득자 또는 부소득자의 실직으로 소득을 상실한 경우  지자체 조례로 정한 사유가 발생한 경우  소득활동 미미(가구원 간호, 간병, 양육), 기초수급 중지, 미결정, 수도·가스 중단, 사회보험료·주택임차료 체납 등  관할', metadata={'source': 'markdowndb\\01_생계_지원\\01_긴급복지_지원제도.md'})]

In [27]:
# vector_store.get()

In [33]:
def test_method(**kwargs):
    print(kwargs)
    return 

kw_dict = {'x': 1, 'y':2}

ans = test_method(**kw_dict)
ans

{'x': 1, 'y': 2}


### Using my class

langchain 으로 서치

In [28]:
from chromaVectorStore import ChromaVectorStore

import torch
from langchain.vectorstores.chroma import Chroma
from langchain.embeddings import SentenceTransformerEmbeddings

In [29]:
# base_model = 'D:\python_project\RAG\Seed_100_tok_epoch_8_final'
base_model = "BM-K/KoSimCSE-roberta-multitask"

In [30]:
emb_info_dict = {'model_name': base_model, 'model_kwargs': {'device': "cuda" if torch.cuda.is_available() else "cpu"},
    'encode_kwargs': {'normalize_embeddings': True}}

lc_emb_func = SentenceTransformerEmbeddings(**emb_info_dict)

In [32]:
vs_info_dict = {'model_name': base_model, 'collection_name': 'langchain', 'persist_directory': 'workspace/chroma_storage', 'embedding_function': lc_emb_func}

vector_store = ChromaVectorStore(**vs_info_dict)

In [33]:
vector_store.vs._collection.count()

1817

In [35]:
vector_store.vs.get().keys()

dict_keys(['ids', 'embeddings', 'metadatas', 'documents', 'uris', 'data'])

In [36]:
vs_dict = vector_store.vs.get()

In [40]:
with open('for_test/contents.txt', 'w', encoding='utf-8') as fw:
    fw.write(f"\n{'-'*50}\n".join(vs_dict['documents']))

In [40]:
q = '졸라 배고파'

res = vector_store.retrieve(q)

In [41]:
for a in res:
    print(a)
    print('-'*20)

[Document(page_content='식비 \n - 1일 5시간 이상(월평균 100시간 이상)인 과정 참여자 중 훈련기관에서 식사를 제공하지 않는 경우 월 최대 6만 6,000 원(3,300 원/일) 지급 - 통학생 중 개인적 사정(식이조절 등으로 인한 개별 도시락 지참 등)으로 식사 제공 대신 식비 지급을 희망할 경우 월 최대 6만 6,000 원(3,300 원/일) 지급'), Document(page_content='영양플러스\n\n대상\n\n소득인정액이 기준 중위소득 80%(4인 기준 432만 1,000원) 이하 가구 중 영양위험요인 (빈혈, 저체중, 성장부진, 영양섭취불량 등)을 가진 임신부, 출산·수유부, 만 6세 이하 영유아\n\n내용\n\n개인별 영양 상태에 따라 영양보충식품(쌀, 감자, 달걀, 우유, 검정콩 등) 제공'), Document(page_content='위기 상황'), Document(page_content='구금시설 수용 등으로 소득을 상실한 경우 ② 중한 질병 또는 부상을 당한 경우 ③ 가구 구성원으로부터 방임 또는 유기되거나 학대 등을 당한 경우 ④ 가족폭력을 당하여 가구구성원과 함께 원만한 가정 생활을 하기 곤란하거나 가구구성원으로부터 성폭력을 당한 경우 ⑤ 화재 또는 자연재해 등으로 인해 거주하는 주택 또는 건물에서 생활하기 곤란한 경우 ⑥ 주소득자')]
--------------------
[Document(page_content='식비 \n - 1일 5시간 이상(월평균 100시간 이상)인 과정 참여자 중 훈련기관에서 식사를 제공하지 않는 경우 월 최대 6만 6,000 원(3,300 원/일) 지급 - 통학생 중 개인적 사정(식이조절 등으로 인한 개별 도시락 지참 등)으로 식사 제공 대신 식비 지급을 희망할 경우 월 최대 6만 6,000 원(3,300 원/일) 지급'), Document(page_content='영양플러스\n\n대상\n\n소득인정액이 기준 중위소득 80%(4인 기준 432만 1,000원) 이하 가구 중 영양

In [1]:
from chromaVectorStore import ChromaVectorStore

In [28]:
base_model = "BM-K/KoSimCSE-roberta-multitask"
vs_info_dict = {'model_name': base_model, 'collection_name': 'langchain', 'persist_directory': 'workspace/chroma_storage'}
vector_store = ChromaVectorStore(**vs_info_dict)


In [9]:
query_text = '배고파ㅏㅏㅏ'

base_model = "BM-K/KoSimCSE-roberta-multitask"
vs_info_dict = {'model_name': base_model, 'collection_name': 'wf_schema', 'persist_directory': './chroma_storage'}
vector_store_wf = ChromaVectorStore(**vs_info_dict)

vector_store.retrieve(query_text)

[[], []]

In [ ]:
vs_info_dict = {'model_name': base_model, 'collection_name': 'langchain', 'persist_directory': './chroma_storage'}
vs2 = ChromaVectorStore(**vs_info_dict)


### DBLoader 이용하기

In [1]:
from workspace.mdLoader import BaseDBLoader
import re
import pickle

In [2]:
doc_loader = BaseDBLoader('workspace/markdownDB')
docs = doc_loader.load(is_split=False, is_regex=False)

len(docs)

c:\Users\jjkha\AppData\Local\Programs\Python\Python310\lib\site-packages\unstructured\documents\html.py:498: FutureWarning: The behavior of this method will change in future versions. Use specific 'len(elem)' or 'elem is not None' test instead.
  rows = body.findall("tr") if body else []


464

In [3]:
print(docs[0])
# 139 376

page_content='LPG용기 사용가구 시설개선\n\n대상\n\nLPG용기 사용 주택에서 LPG 고무호스를 사용 중인 가구\n\n내용\n\nLP가스 고무호스 교체(금속배관) 및 안전장치(퓨즈콕) 등 가스시설 설치 지원 (시공비 약 25만 원 중 20만 원 상당 지원, 자부담 5만 원)\n\n방법\n\n수혜대상자가 소재지 시군구 가스담당부서 또는 읍면동 주민센터(행정복지센터) 방문 신청\n\n문의\n\n소재지 시군구 가스담당부서 또는 읍면동 주민센터(행정복지센터)\n\n한국가스안전공사(☎1544-4500)' metadata={'source': 'workspace\\markdownDB\\01_생계_지원\\01_LPG용기_사용가구_시설개선.md'}


### SentenceTransformersTokenTextSplitter 확인

In [4]:
from langchain.text_splitter import SentenceTransformersTokenTextSplitter

In [5]:
s_splitter = SentenceTransformersTokenTextSplitter(chunk_overlap=10, model_name="BM-K/KoSimCSE-roberta-multitask", tokens_per_chunk=100)

In [6]:
splitted_docs = s_splitter.split_documents(docs)

In [7]:
len(splitted_docs)

1627

In [20]:
print(splitted_docs[0].page_content)
print(splitted_docs[1].page_content)
print(s_splitter.count_tokens(text="행정복지센터 ) 방문 신청 문의 소재지 시군구"))

LPG용기 사용가구 시설개선 대상 LPG용기 사용 주택에서 LPG 고무호스를 사용 중인 가구 내용 LP가스 고무호스 교체 ( 금속배관 ) 및 안전장치 ( 퓨즈콕 ) 등 가스시설 설치 지원 ( 시공비 약 25만 원 중 20만 원 상당 지원, 자부담 5만 원 ) 방법 수혜대상자가 소재지 시군구 가스담당부서 또는 읍면동 주민센터 ( 행정복지센터 ) 방문 신청 문의 소재지 시군구
행정복지센터 ) 방문 신청 문의 소재지 시군구 가스담당부서 또는 읍면동 주민센터 ( 행정복지센터 ) 한국가스안전공사 ( [UNK] - 4500 )
12


In [21]:
from transformers import AutoModel, AutoTokenizer

In [22]:
tokenizer_base = AutoTokenizer.from_pretrained("BM-K/KoSimCSE-roberta-multitask")


In [24]:
len(tokenizer_base.tokenize(splitted_docs[0].page_content))

100

### 텍스트 내용 전처리 테스트

In [3]:
idx = 376
title = re.search('^(.+)\n', only_texts[idx]).groups()[0].strip()
if re.search('대상\n', only_texts[idx]) and re.search('내용\n', only_texts[idx]):
    targets = re.search('대상((.|[\s\n])+?)내용', only_texts[idx]).groups()[0].strip()
else:
    targets = ''
if re.search('내용\n', only_texts[idx]) and re.search('방법\n', only_texts[idx]):
    contents = re.search('내용((.|[\s\n])+?)방법', only_texts[idx]).groups()[0].strip()
else:
    contents = ''

print(title, targets, contents, sep='\n--------------------\n')

햇살론 뱅크
--------------------

--------------------



In [4]:
only_texts = [d.page_content for d in docs_dup]

In [6]:
with open("for_test/only_texts.pickle","wb") as fw:
	pickle.dump(only_texts, fw)

In [2]:
import re
import pickle

with open("for_test/only_texts.pickle","rb") as fr:
    only_texts = pickle.load(fr)

In [40]:
def make_compact_contents(title, targets, contents):
    full_text = f"다음은 '{title}'에 대한 설명입니다.\n\n"
    if targets:
        full_text = full_text + f"먼저, 이 제도의 대상은 다음과 같습니다.\n\n{targets}\n\n"
    if contents:
        full_text = full_text + f"그리고 내용은 다음과 같습니다.\n\n{contents}"
    return full_text

In [84]:
nums = []
for idx, doc in enumerate(docs):
    nums.append([idx, len(doc.page_content)])

In [92]:
# print(docs[16].page_content)

In [85]:
nums

[[0, 199],
 [1, 386],
 [2, 440],
 [3, 294],
 [4, 210],
 [5, 1273],
 [6, 2379],
 [7, 35],
 [8, 199],
 [9, 1735],
 [10, 445],
 [11, 137],
 [12, 1916],
 [13, 350],
 [14, 435],
 [15, 233],
 [16, 2188],
 [17, 336],
 [18, 287],
 [19, 212],
 [20, 303],
 [21, 162],
 [22, 306],
 [23, 162],
 [24, 213],
 [25, 185],
 [26, 691],
 [27, 321],
 [28, 198],
 [29, 262],
 [30, 377],
 [31, 167],
 [32, 657],
 [33, 529],
 [34, 225],
 [35, 268],
 [36, 484],
 [37, 348],
 [38, 455],
 [39, 2459],
 [40, 251],
 [41, 221],
 [42, 252],
 [43, 155],
 [44, 210],
 [45, 253],
 [46, 649],
 [47, 468],
 [48, 1007],
 [49, 706],
 [50, 284],
 [51, 531],
 [52, 196],
 [53, 177],
 [54, 477],
 [55, 630],
 [56, 398],
 [57, 314],
 [58, 148],
 [59, 267],
 [60, 326],
 [61, 302],
 [62, 405],
 [63, 136],
 [64, 250],
 [65, 276],
 [66, 242],
 [67, 691],
 [68, 304],
 [69, 509],
 [70, 361],
 [71, 491],
 [72, 1092],
 [73, 734],
 [74, 479],
 [75, 185],
 [76, 648],
 [77, 569],
 [78, 745],
 [79, 369],
 [80, 451],
 [81, 136],
 [82, 308],
 [83, 3

In [41]:
except_idxes = []

for idx, doc in enumerate(docs):
    all_text = doc.page_content
    try:
        title = re.search('^(.+)\n', all_text).groups()[0].strip()
        # print("0")
        if re.search('대상\n', all_text) and re.search('내용\n', all_text):
            targets = re.search('대상((.|[\s\n])+?)내용', all_text).groups()[0].strip()
        else:
            targets = ""
        # print("1")
        if re.search('내용\n', all_text) and re.search('방법\n', all_text):
            contents = re.search('내용((.|[\s\n])+?)방법', all_text).groups()[0].strip()
        else:
            contents = ""
        # print("2")
        doc.page_content = make_compact_contents(title, targets, contents)
    except:
        except_idxes.append(idx)


In [42]:
print(*except_idxes)

### Chroma DB 로 바로 확인하기!

In [ ]:
from langchain.vectorstores.chroma import Chroma
from langchain.embeddings import SentenceTransformerEmbeddings as STE

In [23]:
base_model = "BM-K/KoSimCSE-roberta-multitask"

In [43]:
emb_info_dict = {'model_name': base_model, 'model_kwargs': {'device': "cpu"},
'encode_kwargs': {'normalize_embeddings': True}}
emb = STE(**emb_info_dict)

In [44]:
vs = Chroma.from_documents(docs, embedding=emb, persist_directory="workspace/chroma_storage")
vs._collection.count()

464

In [35]:
# vs.delete_collection()
# vs.persist()

In [143]:
q1 = "차상위 계층이고 당뇨가 있을 때, 어떤 지원을 받을 수 있을까?"
res1 = [d.page_content for d in vs.max_marginal_relevance_search(q1, k=20)]
res1

["다음은 '장애인 거주시설 실비입소 이용료 지원'에 대한 설명입니다.먼저, 이 제도의 대상은 다음과 같습니다.\n\n\n\n실비 입소 장애인 거주시설에 입소한 장애인 중 기준 중위소득 이하인 장애인\n    ※ 기초생활수급자(생계, 의료, 주거, 교육) 및 차상위계층(본인부담경감대상, 자활, 장애인, 계층확인)은 별도 소득 및 재산조사 없이 지원대상으로 인정\n\n\n\n그리고 내용은 다음과 같습니다.\n\n\n\n장애인거주시설 이용료 중 매월 29만 4천 원 지원\n\n",
 "다음은 '장애인연금'에 대한 설명입니다.먼저, 이 제도의 대상은 다음과 같습니다.\n\n\n\n만 18세 이상의 장애인연금법상 중증장애인 중 본인과 배우자의 소득인정액이 선정기준액 이하에 해당하는 자\n*장애인연금법상 중증장애인: 보건복지부 장관이 정한 요건을 갖춘 심한 장애인\n\n\n\n그리고 내용은 다음과 같습니다.\n\n\n\n근로능력 감소로 소득이 적고, 장애로 인한 추가 지출이 있는 중증 장애인에게 지원\n\n생계급여·의료급여수급자 \n 월 40만 3,180 원 • 기초급여 32만 3,180 원 • 부가급여  8만 원 \n 월 40만 3,180 원 •부가급여 40만3,180원 \n \n 주거급여·교육급여수급자,차상위계층* \n 최고 월 39만 3,180 원  • 기초급여  32만 3,180 원 • 부가급여 7만 원 \n 월 7만 원  •부가급여  7만 원 \n \n 차상위 초과 \n 최고 월 34만 3,180 원 • 기초급여 32만 3,180 원 • 부가급여  2만 원 \n 월 4만 원 • 부가급여  4만 원\n\n차상위계층: 2023년 기준 중위소득 50%(4인 기준 270만 482원) 이하\n\n차상위계층: 2023년 기준 중위소득 50%(4인 기준 270만 482원) 이하\n\n",
 "다음은 '건강보험 차상위'에 대한 설명입니다.먼저, 이 제도의 대상은 다음과 같습니다.\n\n\n\n희귀난치성 및 중증질환자, 6개월 이상 치료를 받고 있거나 치료가 필요한 만성질환자

In [141]:
print(res1[3])

다음은 '건강보험 차상위'에 대한 설명입니다.먼저, 이 제도의 대상은 다음과 같습니다.



희귀난치성 및 중증질환자, 6개월 이상 치료를 받고 있거나 치료가 필요한 만성질환자, 18세 미만 아동·청소년 중 소득인정액이 기준 중위소득 50%(4인 기준 270만 482원) 이하이고 부양요건*을 충족하는 사람

부양요건: 부양의무자가 없거나, 있어도 부양능력이 없거나 부양을 받을 수 없는 경우



그리고 내용은 다음과 같습니다.



요양급여비용 중 본인부담금 경감

희귀난치성·중증질환자 
 요양급여비(입원·외래) 면제, 기본식대의 20%만 부담 
 
 만성질환자, 18세 미만 아동 
 입원비용의 14%, 식대의 20%, 외래비용의 14%(정액 1,000원, 1,500원)만 부담

지역가입자의 건강보험료 지급


